# 캐싱(Casching)
- 동일한 질문에 대한 답변을 여러번 요청하는 경우, LLM공급자에 대한 API호출 횟수를 줄여 비용을 절감할 수 있다.
- API호출 횟수를 줄여, 어플리케이션의 속도를 높힐 수 있다.

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

In [3]:
load_dotenv()

True

In [8]:
llm = ChatOpenAI(model="gpt-4o-mini")

template = """
{country}에 대해서 200자 내외로 요약해줘
"""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm

In [9]:
%%time
response = chain.invoke({"country":"한국"})
print(response.content)

한국은 동아시아에 위치한 국가로, 한반도의 남쪽에 위치합니다. 서울이 수도이며, 경제, 문화, 기술 분야에서 빠르게 발전하고 있습니다. 한식, K-팝, 한국 드라마 등이 세계적으로 인기를 끌고 있으며, 전통과 현대가 조화롭게 공존하는 사회입니다. 한국은 교육열이 높고, IT 산업이 발달하여 글로벌 경제에서 중요한 역할을 하고 있습니다.
CPU times: total: 15.6 ms
Wall time: 2.41 s


# InMemoryCache
- 인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환

- 메모리 공간을 활용한 캐싱을 진행한다. `InMemoryCache` 은 휘발성이 있는 메모리다. 프로그램을 종료하면 날라간다.

- 사용자별로 캐싱을 해야 할 때는 `InMemoryCache`가 유리할 수 있다.

In [10]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용
set_llm_cache(InMemoryCache())

# 체인을 실행
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 반도국으로, 한반도를 차지하고 있습니다. 남한과 북한으로 나뉘어 있으며, 서울은 남한의 수도입니다. 한국은 고유의 문화와 전통, K-팝, K-드라마 등으로 세계적으로 유명합니다. 경제적으로는 기술과 제조업이 발달했으며, 교육 수준이 높습니다. 한국의 역사적 유산과 자연 경관도 많은 관광객을 끌어옵니다.
CPU times: total: 547 ms
Wall time: 2.65 s


In [12]:
%%time
# 시간이 엄청 줄었다! 비용역시 발생하지 않는다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 반도국으로, 한반도를 차지하고 있습니다. 남한과 북한으로 나뉘어 있으며, 서울은 남한의 수도입니다. 한국은 고유의 문화와 전통, K-팝, K-드라마 등으로 세계적으로 유명합니다. 경제적으로는 기술과 제조업이 발달했으며, 교육 수준이 높습니다. 한국의 역사적 유산과 자연 경관도 많은 관광객을 끌어옵니다.
CPU times: total: 0 ns
Wall time: 1.99 ms


# SQLite Cache
- 휘발성이 없애기 위해, DB에 저장하기 위함이다.

In [13]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [14]:
%%time
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 한반도의 국가로, 남북으로 나뉘어 있습니다. 남한은 민주주의 국가로 경제가 발달하였으며, K-팝, K-드라마 등 문화가 세계적으로 인기를 끌고 있습니다. 북한은 사회주의 국가로, 정치적 특성이 다릅니다. 한국은 풍부한 역사와 전통, 맛있는 음식, 아름다운 자연 경관을 자랑합니다.
CPU times: total: 46.9 ms
Wall time: 2.65 s
